# Fairness and Explainability with SageMaker Clarify using AWS SDK for Python (Boto3)

## Runtime

This notebook takes approximately 30 minutes to run.

## Contents

1. [Overview](#Overview)
1. [Prerequisites and Data](#Prerequisites-and-Data)
    1. [Import libraries](#Import-libraries)
    1. [Set configurations](#Set-configurations)
    1. [Download data](#Download-data)
    1. [Loading the data: Adult Dataset](#Loading-the-data:-Adult-Dataset) 
    1. [Data inspection](#Data-inspection) 
    1. [Data encoding and upload to S3](#Encode-and-Upload-the-Data) 
1. [Train and Deploy XGBoost Model](#Train-XGBoost-Model)
    1. [Train Model](#Train-Model)
    1. [Create Model](#Create-Model)
1. [Amazon SageMaker Clarify](#Amazon-SageMaker-Clarify)
    1. [Set configurations](#Set-configurations)
    1. [Get Started with a SageMaker Clarify Container](#Get-Started-with-a-SageMaker-Clarify-Container)
    1. [Configure a SageMaker Clarify Processing Job Container's Input and Output Parameters ](#Configure-a-SageMaker-Clarify-Processing-Job-Container's-Input-and-Output-Parameters)
    1. [Configure Analysis Config](#Configure-Analysis-Config)
    1. [Run Sagemaker Clarify Processing Job](#Run-Sagemaker-Clarify-Processing-Job)
    1. [Viewing Clarify Report](#Viewing-Clarify-Report)
        1. [Viewing the Bias Report](#Viewing-the-Bias-Report)
        1. [Viewing the Explainability Report](#Viewing-the-Explainability-Report)
        1. [Analysis of local explanations](#Analysis-of-local-explanations)
1. [Clean Up](#Clean-Up)

## Overview
Amazon SageMaker Clarify helps improve your machine learning models by detecting potential bias and helping explain how these models make predictions. The fairness and explainability functionality provided by SageMaker Clarify takes a step towards enabling AWS customers to build trustworthy and understandable machine learning models. The product comes with the tools to help you with the following tasks.

* Measure biases that can occur during each stage of the ML lifecycle (data collection, model training and tuning, and monitoring of ML models deployed for inference).
* Generate model governance reports targeting risk and compliance teams and external regulators.
* Provide explanations of the data, models, and monitoring used to assess predictions.

This sample notebook walks you through:  
1. Key terms and concepts needed to understand SageMaker Clarify
1. Measuring the pre-training bias of a dataset and post-training bias of a model
1. Explaining the importance of the various input features on the model's decision
1. Accessing the reports through SageMaker Studio if you have an instance set up.

In doing so, the notebook first trains a [SageMaker XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) model using training dataset, then use SageMaker Clarify to analyze a testing dataset in CSV format. SageMaker Clarify also supports analyzing dataset in [SageMaker JSON Lines dense format](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html#common-in-formats), which is illustrated in [another notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability_jsonlines_format.ipynb). This notebook uses AWS SDK for Python (Boto3) for clarify  processing API calls, making the same API calls using Amazon SageMaker Python SDK is illustrated in [this notebook](https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb).

## Prerequisites and Data

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import boto3
import time
from datetime import datetime
from sagemaker import get_execution_role, session

### Set configurations

In [ ]:
# Initialize sagemaker session
sagemaker_session = session.Session(boto3.Session())

region = boto3.session.Session().region_name
print(f"Region: {region}")

role = get_execution_role()
print(f"Role: {role}")

s3_client = boto3.client("s3")
bucket = sagemaker_session.default_bucket()

prefix = "sagemaker/DEMO-sagemaker-clarify"

### Download data
Data Source: [https://archive.ics.uci.edu/ml/machine-learning-databases/adult/](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/)

Let's __download__ the data and save it in the local folder with the name adult.data and adult.test from UCI repository$^{[2]}$.

$^{[2]}$Dua Dheeru, and Efi Karra Taniskidou. "[UCI Machine Learning Repository](http://archive.ics.uci.edu/ml)". Irvine, CA: University of California, School of Information and Computer Science (2017).

In [ ]:
adult_columns = [
    "Age",
    "Workclass",
    "fnlwgt",
    "Education",
    "Education-Num",
    "Marital Status",
    "Occupation",
    "Relationship",
    "Ethnic group",
    "Sex",
    "Capital Gain",
    "Capital Loss",
    "Hours per week",
    "Country",
    "Target",
]
if not os.path.isfile("adult.data"):
    s3_client.download_file(
        "sagemaker-sample-files", "datasets/tabular/uci_adult/adult.data", "adult.data"
    )
    print("adult.data saved!")
else:
    print("adult.data already on disk.")

if not os.path.isfile("adult.test"):
    s3_client.download_file(
        "sagemaker-sample-files", "datasets/tabular/uci_adult/adult.test", "adult.test"
    )
    print("adult.test saved!")
else:
    print("adult.test already on disk.")

### Loading the data: Adult Dataset
From the UCI repository of machine learning datasets, this database contains 14 features concerning demographic characteristics of 45,222 rows (32,561 for training and 12,661 for testing). The task is to predict whether a person has a yearly income that is more or less than $50,000.

Here are the features and their possible values:
1. **Age**: continuous.
1. **Workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
1. **Fnlwgt**: continuous (the number of people the census takers believe that observation represents).
1. **Education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
1. **Education-num**: continuous.
1. **Marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
1. **Occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
1. **Relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
1. **Ethnic group**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
1. **Sex**: Female, Male.
    * **Note**: this data is extracted from the 1994 Census and enforces a binary option on Sex
1. **Capital-gain**: continuous.
1. **Capital-loss**: continuous.
1. **Hours-per-week**: continuous.
1. **Native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Next, we specify our binary prediction task:  
15. **Target**: <=50,000, >$50,000.

In [ ]:
training_data = pd.read_csv(
    "adult.data", names=adult_columns, sep=r"\s*,\s*", engine="python", na_values="?"
).dropna()

testing_data = pd.read_csv(
    "adult.test", names=adult_columns, sep=r"\s*,\s*", engine="python", na_values="?", skiprows=1
).dropna()

training_data.head()

### Data inspection
Plotting histograms for the distribution of the different features is a good way to visualize the data. Let's plot a few of the features that can be considered _sensitive_.  
Let's take a look specifically at the Sex feature of a census respondent. In the first plot we see that there are fewer Female respondents as a whole but especially in the positive outcomes, where they form ~$\frac{1}{7}$th of respondents.

In [ ]:
%matplotlib inline
training_data["Sex"].value_counts().sort_values().plot(kind="bar", title="Counts of Sex", rot=0)

In [ ]:
training_data["Sex"].where(training_data["Target"] == ">50K").value_counts().sort_values().plot(
    kind="bar", title="Counts of Sex earning >$50K", rot=0
)

### Encode and Upload the Dataset
Here we encode the training and test data. Encoding input data is not necessary for SageMaker Clarify, but is necessary for the model.

In [ ]:
from sklearn import preprocessing


def number_encode_features(df):
    result = df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column].fillna("None"))
    return result, encoders


training_data = pd.concat([training_data["Target"], training_data.drop(["Target"], axis=1)], axis=1)
training_data, _ = number_encode_features(training_data)
training_data.to_csv("train_data.csv", index=False, header=False)

testing_data, _ = number_encode_features(testing_data)
test_features = testing_data.drop(["Target"], axis=1)
test_target = testing_data["Target"]
test_features.to_csv("test_features.csv", index=False, header=False)

A quick note about our encoding: the "Female" Sex value has been encoded as 0 and "Male" as 1.

In [ ]:
training_data.head()

Lastly, let's upload the data to S3.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload("train_data.csv", "s3://{}/{}".format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type="csv")
test_uri = S3Uploader.upload("test_features.csv", "s3://{}/{}".format(bucket, prefix))

### Train XGBoost Model
#### Train Model
Since our focus is on understanding how to use SageMaker Clarify, we keep it simple by using a standard XGBoost model. For this section we will be using Amazon SageMaker Python SDK for simplicity.

In [ ]:
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator

# This is references the AWS managed XGBoost container
xgboost_image_uri = retrieve(region=region, framework="xgboost", version="1.5-1")

xgb = Estimator(
    xgboost_image_uri,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    disable_profiler=True,
    sagemaker_session=sagemaker_session,
)

xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective="binary:logistic",
    num_round=800,
)

xgb.fit({"train": train_input}, logs=False)

#### Create Model
Here we create the SageMaker model.

In [ ]:
model_name = "DEMO-clarify-model-{}".format(datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)

## Amazon SageMaker Clarify
Now that you have your model set up, let's say hello to SageMaker Clarify! We will be using AWS SDK for Python (Boto3) for all API calls here. To get an understanding of how SageMaker Clarify Processing job works refer [here](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-how-it-works.html).

### Set configurations

In [ ]:
# Initialise Sagemaker boto3 client
sagemaker_client = boto3.Session().client("sagemaker")

# Note: We will be using role fetched in section 1 for convenient/demo purpose so that the notebook
# can be easily executed in SageMaker Studio or SageMaker Notebook Instance.
# You can use their own execution role for their project.
print(f"Role: {role}")

print(f"Region: {region}")

### Get Started with a SageMaker Clarify Container
Amazon SageMaker provides prebuilt SageMaker Clarify container images that include the libraries and other dependencies needed to compute bias metrics and feature attributions for explainability. This image has been enabled to run SageMaker [Process Data](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html) in your account.

Region wise adresses of Clarify container images can be referred [here](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-container.html).

In [ ]:
# Using retrieve API of Amazon SageMaker Python SDK for demo purposes,
# refer list of image URIs to if you wish to skip using this API.
clarify_image_uri = retrieve(region=region, framework="clarify", version="1.0")
print(f"Clarify Image URI: {clarify_image_uri}")

### Detecting Bias
SageMaker Clarify helps you detect possible pre- and post-training biases using a variety of metrics.

#### Configure a SageMaker Clarify Processing Job Container's Input and Output Parameters 
The Processing Job requires that you specify the following input parameters: a dataset files with input name "dataset" as Amazon S3 object or prefix, and an analysis configuration file with input name "analysis_config" as an Amazon S3 object. The job also requires an output parameter: the output location as an Amazon S3 prefix.

In [ ]:
bias_analysis_config_path = "s3://{}/{}/bias_analysis_config.json".format(bucket, prefix)
analysis_result_path = "s3://{}/{}/output".format(bucket, prefix)

#### Configure Analysis Config
The inputs for the analysis are configured by the parameters of the ProcessingInput API. The "analysis_config" value of the input_name specifies the JSON file that contains the configuration values. The path to the JSON file is provided in the source parameter of ProcessingInput. More details on configuring an analysis config can be referred [here](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html).

For our example use case we will be using the following analysis config:

In [ ]:
!echo
!cat bias_analysis_config.json

`bias_analysis_config.json` here contains configuration values for detecting bias using a clarify container:
* `dataset_type` specify the format of your dataset, for thsi example as we are using csv dataset this will be `text/csv`
* `headers` is the list of column names in the dataset
* `label` specified the target attribute for the model to be used for bias metrics. Specified as a column name, or as index if the dataset format is CSV
* SageMaker Clarify also needs information on what the sensitive columns (`facets`) are, what the sensitive features (`facet: values_or_threshold`) may be, and what the desirable outcomes are (`label_values_or_threshold`).
SageMaker Clarify can handle both categorical and continuous data for `facet: values_or_threshold` and for `label_values_or_threshold`. In this case we are using categorical data.
* `group_variable` A column name or index to indicate the group variable to be used for the *bias metric Conditional Demographic Disparity*
* `probability_threshold` is to indicate the threshold to select the binary label in the case of binary classification. XGBoost model outputs probabilities of samples, so SageMaker Clarify invokes the endpoint then uses `probability_threshold` to convert the probability to binary labels for bias analysis. Prediction above the threshold is interpreted as label value `1` and below or equal as label value `0`.
* `method` is the list of methods and their parameters for the analyses and reports. If any section is omitted, then it is not computed.
  * **Pre-training Bias**: Bias can be present in your data before any model training occurs. Inspecting your data for bias before training begins can help detect any data collection gaps, inform your feature engineering, and help you understand what societal biases the data may reflect.

    Computing pre-training bias metrics does not require a trained model. One can selectively choose metrics to be calculated and added to report by using a specific `methods` list in this attribute.
  * **Post-training Bias**: Computing post-training bias metrics does require a trained model.

    Unbiased training data (as determined by concepts of fairness measured by bias metric) may still result in biased model predictions after training. Whether this occurs depends on several factors including hyperparameter choices. One can selectively choose metrics to be calculated and added to report by using a specific `methods` list in this attribute.
* `instance_type` and `instance_count` specify your preferred instance type and instance count used to run your model on during SageMaker Clarify's processing. The testing dataset is small so a single standard instance is good enough to run this example. If your have a large complex dataset, you may want to use a better instance type to speed up, or add more instances to enable Spark parallelization.
* `accept_type` denotes the endpoint response payload format, and `content_type` denotes the payload format of request to the endpoint. As per the example model we created above both of these will be `text/csv`

In [ ]:
# Upload the analysis_config to the concerned S3 path.
S3Uploader.upload("bias_analysis_config.json", "s3://{}/{}".format(bucket, prefix))

#### Run Sagemaker Clarify Processing Job
Refer this documentation to [configure a clarify procesing job](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-parameters.html) for your use case.

In [ ]:
def create_processing_job(analysis_config_path):
    processing_job_name = "DEMO-clarify-job-{}".format(datetime.now().strftime("%d-%m-%Y-%H-%M-%S"))

    response = sagemaker_client.create_processing_job(
        ProcessingJobName=processing_job_name,
        AppSpecification={"ImageUri": clarify_image_uri},
        ProcessingInputs=[
            {
                "InputName": "analysis_config",
                "S3Input": {
                    "S3DataType": "S3Prefix",
                    "S3InputMode": "File",
                    "S3Uri": analysis_config_path,
                    "LocalPath": "/opt/ml/processing/input/config",
                    "S3CompressionType": "None",
                },
            },
            {
                "InputName": "dataset",
                "S3Input": {
                    "S3DataType": "S3Prefix",
                    "S3InputMode": "File",
                    "S3Uri": train_uri,
                    "LocalPath": "/opt/ml/processing/input/data",
                    "S3CompressionType": "None",
                },
            },
        ],
        ProcessingOutputConfig={
            "Outputs": [
                {
                    "OutputName": "analysis_result",
                    "S3Output": {
                        "S3Uri": analysis_result_path,
                        "LocalPath": "/opt/ml/processing/output",
                        "S3UploadMode": "EndOfJob",
                    },
                }
            ]
        },
        ProcessingResources={
            "ClusterConfig": {
                "InstanceCount": 1,
                "InstanceType": "ml.m5.xlarge",
                "VolumeSizeInGB": 30,
            }
        },
        StoppingCondition={
            "MaxRuntimeInSeconds": 3600,
        },
        RoleArn=role,
    )

    return processing_job_name

Here is a brief explanation of inputs used above, for detailed documentation check [CreateProcessingJob API reference](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateProcessingJob.html):
* AppSpecification: Here we provide the region specific clarify image uri we fethced earlier
* ProcessingInputs: Clarify job requires that you provde two ProcessingInput parameters.
  * The analysis configuration JSON file for a SageMaker Clarify job must be specified as an Amazon S3 object with the InputName "analysis_config". We will be providing the example analysis_configs that we have provided with this notebook. 
  * InputName dataset which we are providing here as an Amazon S3 object.
* ProcessingOutputConfig: The job also requires an output parameter, the output location as an Amazon S3 prefix with the OutputName "analysis_result". The S3UploadMode should be set to "EndOfJob", because the analysis results is generated at the end of the job. We will be providing here the `analysis_result_path` that we configured earlier.
* ProcessingResources contains the ClusterConfig specifying the ML compute instance_type we want to use and the count. SageMaker SHAP analysis is CPU-intensive, using a better instance type should speed up the analysis. The SageMaker Clarify job doesn’t use GPU.
* StoppingCondition: Using a maximum limit of 30 min for example job run. You can set the MaxRuntimeInSeconds of a SageMaker Clarify job to up to 7 days (604800 seconds). If the job cannot be completed within this time limit, it will be force-stopped and no analysis results are provided.

In [ ]:
processing_job_name = create_processing_job(bias_analysis_config_path)

# Wait for processing job to complete
status = sagemaker_client.describe_processing_job(ProcessingJobName=processing_job_name)

while status["ProcessingJobStatus"] == "InProgress":
    status = sagemaker_client.describe_processing_job(ProcessingJobName=processing_job_name)
    print(status["ProcessingJobStatus"])
    time.sleep(60)

#### Viewing the Bias Report
In Studio, you can view the results under the experiments tab.

<img src="./recordings/bias_report.gif">

Each bias metric has detailed explanations with examples that you can explore.

<img src="./recordings/bias_detail.gif">

You could also summarize the results in a handy table!

<img src="./recordings/bias_report_chart.gif">

If you're not a Studio user yet you can access the complete analysis report at the following S3 bucket.


In [ ]:
analysis_result_path

### Explaining Predictions
There are expanding business needs and legislative regulations that require explanations of _why_ a model made the decision it did. SageMaker Clarify uses SHAP to explain the contribution that each input feature makes to the final decision.

#### Configure a SageMaker Clarify Processing Job Container's Input and Output Parameters 

In [ ]:
explainability_analysis_config_path = "s3://{}/{}/explainability_analysis_config.json".format(
    bucket, prefix
)
analysis_result_path = "s3://{}/{}/output".format(bucket, prefix)

#### Configure Analysis Config
For our example use case we will be using the following analysis config:

In [ ]:
!echo
!cat explainability_analysis_config.json

`explainability_analysis_config.json` here contains configuration values for detecting bias using a clarify container:
* `dataset_type` specify the format of your dataset, for thsi example as we are using csv dataset this will be `text/csv`
* `headers` is the list of column names in the dataset
* `label` specified the target attribute for the model to be used for bias metrics. Specified as a column name, or as index if the dataset format is CSV
* `method` is the list of methods and their parameters for the analyses and reports. If any section is omitted, then it is not computed.
  * **shap:** Kernel SHAP algorithm requires a baseline (also known as background dataset). If not provided, a baseline is calculated automatically by SageMaker Clarify using K-means or K-prototypes in the input dataset. Baseline dataset type shall be the same as `dataset_type`, and baseline samples shall only include features. By definition, `baseline` should either be a S3 URI to the baseline dataset file, or an in-place list of samples. In this case we chose the latter, and put the first sample of the test dataset to the list.

* `instance_type` and `instance_count` specify your preferred instance type and instance count used to run your model on during SageMaker Clarify's processing. The testing dataset is small so a single standard instance is good enough to run this example. If your have a large complex dataset, you may want to use a better instance type to speed up, or add more instances to enable Spark parallelization.
* `accept_type` denotes the endpoint response payload format, and `content_type` denotes the payload format of request to the endpoint. As per the example model we created above both of these will be `text/csv`

In [ ]:
# Upload the analysis_config to the concerned S3 path.
S3Uploader.upload("explainability_analysis_config.json", "s3://{}/{}".format(bucket, prefix))

#### Run Sagemaker Clarify Processing Job

In [ ]:
processing_job_name = create_processing_job(explainability_analysis_config_path)

# Wait for processing job to complete
status = sagemaker_client.describe_processing_job(ProcessingJobName=processing_job_name)

while status["ProcessingJobStatus"] == "InProgress":
    status = sagemaker_client.describe_processing_job(ProcessingJobName=processing_job_name)
    print(status["ProcessingJobStatus"])
    time.sleep(60)

#### Viewing the Explainability Report
As with the bias report, you can view the explainability report in Studio under the experiments tab.


<img src="./recordings/explainability_detail.gif">

The Model Insights tab contains direct links to the report and model insights.

If you're not a Studio user yet you can access the complete analysis report at the following S3 bucket.

In [ ]:
analysis_result_path

#### Analysis of local explanations
It is possible to visualize the the local explanations for single examples in your dataset. You can use the obtained results from running Kernel SHAP algorithm for global explanations.

You can simply load the local explanations stored in your output path, and visualize the explanation (i.e., the impact that the single features have on the prediction of your model) for any single example.

In [ ]:
local_explanations_out = pd.read_csv(analysis_result_path + "/explanations_shap/out.csv")
feature_names = [str.replace(c, "_label0", "") for c in local_explanations_out.columns.to_series()]
local_explanations_out.columns = feature_names

selected_example = 111
print(
    "Example number:",
    selected_example,
    "\nwith model prediction:",
    sum(local_explanations_out.iloc[selected_example]) > 0,
)
print("\nFeature values -- Label", training_data.iloc[selected_example])
local_explanations_out.iloc[selected_example].plot(
    kind="bar", title="Local explanation for the example number " + str(selected_example), rot=90
)

### Clean Up
Finally, don't forget to clean up the resources we set up and used for this demo!

In [ ]:
sagemaker_client.delete_model(ModelName=model_name)